In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import seaborn as sns
from random import randint
from main_data import (
    key_to_camelot,
    open_wheel,
    add_features,
    get_playlist,
    get_friendly,
    pickl,
    unpickl,
    sp
)


In [14]:
# %store -r streams_df streams_af_df no_skip_df playlist_af_df
# streams_df, streams_af_df, no_skip_df, playlist_af_df = streams_df, streams_af_df, no_skip_df, playlist_af_df


In [65]:
# Import DataFrames generated by main_data.py

(
    all_streams_df,
    streams_df,
    streams_af_df,
    no_skip_df,
    playlist_af_df,
    pod_df,
    wheel_df,
) = unpickl(
    "all_streams_df.p",
    "streams_df.p",
    "streams_af_df.p",
    "no_skip_df.p",
    "playlist_af_df.p",
    "podcasts_df.p",
    "wheel_df.p",
)


In [16]:
# missing_tracks = []
# for i in range(len(comparison)):
#     missing_tracks.append(sp.track(comparison.iloc[i])['external_urls'])
# missing_tracks
streams_af_df[streams_af_df.timestamp.astype(str).duplicated() == True].sort_values(by="timestamp")


In [11]:
pd.DataFrame(all_streams_df.groupby("time"))

0        (2019-07-21 04:57:57+00:00, [timestamp])
1        (2019-07-21 05:02:29+00:00, [timestamp])
2        (2019-07-21 05:16:50+00:00, [timestamp])
3        (2019-07-21 05:17:50+00:00, [timestamp])
4        (2019-07-21 05:22:16+00:00, [timestamp])
                           ...                   
85001    (2022-10-22 23:02:17+00:00, [timestamp])
85002    (2022-10-22 23:05:56+00:00, [timestamp])
85003    (2022-10-22 23:06:46+00:00, [timestamp])
85004    (2022-10-22 23:15:26+00:00, [timestamp])
85005    (2022-10-22 23:22:10+00:00, [timestamp])
Length: 85006, dtype: object

In [23]:
# pd.to_timedelta
def list_duplicatetimes(df):
    return df[df.ddate.duplicated() == True].sort_values(by="ddate")

list_duplicatetimes(all_streams_df)


# def format_td(td):
#     """Converts timedelta to hh:mm:ss strictly for presentation purposes"""
#     minutes, seconds = divmod(td.seconds + td.days * 86400, 60)
#     hours, minutes = divmod(minutes, 60)
#     return "{:d}:{:02d}:{:02d}".format(hours, minutes, seconds, td.microseconds)

,timestamp,playtime,track,artist,album,id,episode,show,spotify_episode_uri,start,end,shuffle,ddate,dtime,date,time,month,year,day
57082,2019-07-21 09:35:00+00:00,321,LMK_WHAT’S REALLY GOOD REMIX_ FEAT_PRINCESS NO...,Kelela,"TAKE ME A_PART, THE REMIXES",1q5h9UkGVYVoec4ydU9P0d,None,None,None,trackdone,trackdone,False,2019-07-21,09:35:00,07/21/2019,09:35:00,Jul,2019,Sun
79585,2019-07-21 09:19:50+00:00,269,Turn To Dust,Kelela,Take Me Apart,6TiZRho24MdxqKQKrNITi2,None,None,None,trackdone,trackdone,False,2019-07-21,09:19:50,07/21/2019,09:19:50,Jul,2019,Sun
58923,2019-07-21 16:28:55+00:00,1,Sidepiece,NxWorries,Yes Lawd!,5jq3Lzz8Y4ROTFlOyMQYdW,None,None,None,trackdone,unexpected-exit,False,2019-07-21,16:28:55,07/21/2019,16:28:55,Jul,2019,Sun
68929,2019-07-21 06:35:41+00:00,197,Sticky,Ravyn Lenae,Sticky,1rfGO3N4DPGodJLSSY9Re5,None,None,None,trackdone,trackdone,False,2019-07-21,06:35:41,07/21/2019,06:35:41,Jul,2019,Sun
80381,2019-07-21 08:50:30+00:00,269,All the Way Down,Kelela,Hallucinogen,6JT37WmQRQawN2By1ZQR0N,None,None,None,trackdone,trackdone,False,2019-07-21,08:50:30,07/21/2019,08:50:30,Jul,2019,Sun
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28499,2022-10-22 08:41:30+00:00,216,Ciara,Sudan Archives,Natural Brown Prom Queen,1jSZYE0DyzD8jDke47YZWi,None,None,None,trackdone,trackdone,False,2022-10-22,08:41:30,10/22/2022,08:41:30,Oct,2022,Sat
53680,2022-10-22 08:41:47+00:00,6,A/B Machines,Sleigh Bells,Treats,3RdCbriHX35hNIcF6dIhgM,None,None,None,trackdone,endplay,False,2022-10-22,08:41:47,10/22/2022,08:41:47,Oct,2022,Sat
81775,2022-10-22 06:57:38+00:00,195,Conquer & Divide,El Michels Affair meets Liam Bailey,Ekundayo Inversions,1lx0AGEwgqFHBb9lo3k1Pd,None,None,None,clickrow,trackdone,False,2022-10-22,06:57:38,10/22/2022,06:57:38,Oct,2022,Sat
46048,2022-10-22 07:36:44+00:00,142,FWM (with Yosohn),G Herbo,Survivor’s Remorse: A Side,6FgLgiZfogBrRLUKTL9DpW,None,None,None,clickrow,trackdone,False,2022-10-22,07:36:44,10/22/2022,07:36:44,Oct,2022,Sat


In [71]:
# all_streams_df.loc[all_streams_df[all_streams_df.timestamp].dt.year == "2020"]
all_streams_df.timestamp.dt.year == 2020
    # df["dtime"] = df[["timestamp"]].apply(lambda x: x.dt.time)

# dti = [all_streams_df.timestamp.reset_index(drop=True)]
all_streams_df.set_index(pd.DatetimeIndex(all_streams_df.ddate)).sort_index()

,timestamp,playtime,track,artist,album,id,episode,show,spotify_episode_uri,start,end,shuffle,ddate,dtime,date,time,month,year,day
ddate,,,,,,,,,,,,,,,,,,,
2019-07-21,2019-07-21 09:35:00+00:00,321,LMK_WHAT’S REALLY GOOD REMIX_ FEAT_PRINCESS NO...,Kelela,"TAKE ME A_PART, THE REMIXES",1q5h9UkGVYVoec4ydU9P0d,None,None,None,trackdone,trackdone,False,2019-07-21,09:35:00,07/21/2019,09:35:00,Jul,2019,Sun
2019-07-21,2019-07-21 06:28:42+00:00,207,"After The Storm (feat. Tyler, The Creator & Bo...",Kali Uchis,Isolation,1otG6j1WHNvl9WgXLWkHTo,None,None,None,trackdone,trackdone,False,2019-07-21,06:28:42,07/21/2019,06:28:42,Jul,2019,Sun
2019-07-21,2019-07-21 07:03:12+00:00,244,Miami (feat. BIA),Kali Uchis,Isolation,5oJAaPRS2EzNIc8EWIbucE,None,None,None,trackdone,trackdone,False,2019-07-21,07:03:12,07/21/2019,07:03:12,Jul,2019,Sun
2019-07-21,2019-07-21 06:11:24+00:00,207,Another Lifetime,Nao,Another Lifetime,36YutBq4lokWfRAFXJ6qrF,None,None,None,trackdone,trackdone,False,2019-07-21,06:11:24,07/21/2019,06:11:24,Jul,2019,Sun
2019-07-21,2019-07-21 08:35:53+00:00,257,Enemy (Prod. Nguzunguzu),Kelela,Cut 4 Me (Deluxe),5I8xU4V87ADu9lzf4blX6T,None,None,None,trackdone,trackdone,False,2019-07-21,08:35:53,07/21/2019,08:35:53,Jul,2019,Sun
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-22,2022-10-22 21:20:14+00:00,15,Fuck a War,Geto Boys,We Can't Be Stopped,2kEECRiLCWgUrWkGcmNJes,None,None,None,trackdone,endplay,False,2022-10-22,21:20:14,10/22/2022,21:20:14,Oct,2022,Sat
2022-10-22,2022-10-22 21:09:08+00:00,213,We Can't Be Stopped,Geto Boys,We Can't Be Stopped,4JcblA67lQGf06u0sNVJkS,None,None,None,trackdone,trackdone,False,2022-10-22,21:09:08,10/22/2022,21:09:08,Oct,2022,Sat
2022-10-22,2022-10-22 08:01:29+00:00,190,Haze.Boogie.Life,Mykki Blanco,Cosmic Angel: The Illuminati Prince/ss,4KGsE21p778txzQxUpHVlj,None,None,None,trackdone,trackdone,False,2022-10-22,08:01:29,10/22/2022,08:01:29,Oct,2022,Sat


In [ ]:
# I don't recall listening to this artist so much. Investigating if it was on repeat on accident or something.

why_tala = streams_af_df.query('"Hope Tala" in artist')
why_why_tala = (
    why_tala.groupby(by="date")
    .sum(numeric_only=True)
    .sort_values(by="playtime", ascending=False)
)
why_why_tala["playtime"] = why_why_tala["playtime"].apply(lambda x: x / 60000)
why_why_tala.head(50)


In [ ]:
# Looks like I just let her run on most popular song repeat for a few hours.
streams_af_df.query('date == "11/27/2019"').sort_values(by="timestamp")


In [ ]:
no_skip_df.query('artist.str.contains("(?i)miles davis")', engine="python")


In [ ]:
def top_songs(years="all", skips=False, features=True):
    if skips and features:
        df = streams_af_df
    elif not skips:
        df = df.query("(playtime / duration) > 0.51").reset_index()


pivot_track = pd.DataFrame(
    no_skip_df.query('year == "2022"')
    .pivot_table(columns=["artist", "track", "year"], aggfunc="size")
    .sort_values(ascending=False)
)

pivot_track.head(30)


In [ ]:
no_skip_df.query('year == "2022"')


In [ ]:
# # Check for tracks missing from new df
# series1 = duration_prep['spotify_track_uri']
# series2 = local_features['uri']
# comparison = series1[~series1.isin(series2)]
# comparison


In [ ]:
# # Show missing track URLs for further investigation; track is likely removed or yet to be analyzed for audio_features.
# missing_tracks = []
# for i in range(len(comparison)):
#     missing_tracks.append(sp.track(comparison.iloc[i])['external_urls'])
# missing_tracks


In [ ]:
# Most played tracks
pivot_track = pd.DataFrame(
    no_skip_df.pivot_table(columns=["artist", "track"], aggfunc="size").sort_values(
        ascending=False
    )
)
pivot_track


In [ ]:
# Most listened to artists by number of songs played
pivot_artist = pd.DataFrame(
    no_skip_df.pivot_table(columns=["artist"], aggfunc="size").sort_values(
        ascending=False
    )
)
pivot_artist


In [ ]:
no_skip_df


In [ ]:
miles = no_skip_df.query('artist.str.contains("(?i)Miles Davis")')
pivot_miles = pd.pivot_table(miles, values="playtime", index=["album"], aggfunc="sum")
pivot_miles = pivot_miles.sort_values(by="playtime", ascending=False)
miles


In [ ]:
pivot_album_time = pd.pivot_table(
    no_skip_df, values="playtime", index=["album"], aggfunc="sum"
)


In [ ]:
pivot_album_time = pivot_album_time.sort_values(by="playtime", ascending=False)
pivot_album_time.head(20)


In [ ]:
pivot_artist_time = pd.pivot_table(
    no_skip_df, values="playtime", index=["artist"], aggfunc="sum"
)


In [ ]:
# print(pivot_artist_time.sort_values().to_string())
artist_time = pivot_artist_time.sort_values(
    by="playtime", ascending=False
).reset_index()
print(artist_time.to_string())


In [ ]:
# Artists played more than 10 mins
repeated_artists = artist_time[:1750]
# Number of unique tracks played from those artists

len(streams_af_df["track"].unique())


In [ ]:
still_repeated = repeated_artists["artist"]


In [ ]:
# Count number of unique tracks per artist and create ascending list
still_still_repeated = no_skip_df.query("artist in @still_repeated")
# still_still_repeated.loc[:, 'artist'] = still_still_repeated.copy().loc[:, 'artist'].astype(str)
group = still_still_repeated.groupby("artist")
songs_per_artist = group.apply(lambda x: x["track"].unique())
unique_songs = songs_per_artist.reset_index(name="unique_titles")


In [ ]:
unique_songs["unique_count"] = unique_songs.iloc[:, 1].str.len()


In [ ]:
unique_songs = unique_songs.sort_values(by="unique_count").reset_index()


In [ ]:
unique_export = unique_songs.query("unique_count <= 8")["artist"]


In [ ]:
pivot_artist_time.playtime


In [ ]:
# Artists with 10 or less songs played that I've heard more than an hour of. Indicative of artists I should explore more of.
pivot_artist_time.query("playtime > 3600000").query("artist in @unique_export")


Todo: Next step after the above is to plot least uniques with most ms played

In [ ]:
# Convert
pivot_artist_time["playtime"] = pd.to_timedelta(pivot_artist_time.playtime, unit="ms")


In [ ]:
def format_td(td):
    """Converts timedelta to hh:mm:ss strictly for presentation purposes"""
    minutes, seconds = divmod(td.seconds + td.days * 86400, 60)
    hours, minutes = divmod(minutes, 60)
    return "{:d}:{:02d}:{:02d}".format(hours, minutes, seconds, td.microseconds)


In [ ]:
pivot_artist_time["playtime"] = pivot_artist_time["playtime"].apply(format_td)


In [ ]:
# Show longest songs
no_skip_df.sort_values(by="playtime", axis=0, ascending=False)


In [ ]:
# How to control this for missing months? do october 25 2019 thru 2022
sns_prep = streams_af_df.query('year == "2019"| year == "2020" | year == "2021"')
sns_prep = pd.pivot_table(
    sns_prep,
    values="playtime",
    index=["month"],
    aggfunc="sum",
).reset_index()
sns_prep["hours played"] = sns_prep["playtime"].apply(lambda x: x / 3600000)
sns.barplot(data=sns_prep, x="month", y="hours played")


In [ ]:
# sum of unique uris closely maps to total skips per month
sns_prep = streams_af_df.copy().drop_duplicates(subset="id")
sns_prep["count"] = 1
sns_prep = (
    sns_prep.loc[:, ["month", "duration", "playtime", "shuffle", "count"]]
    .groupby(by="month")
    .sum(numeric_only=True)
)
sns.barplot(data=sns_prep, x=sns_prep.index, y="count")


In [ ]:
# good way to find months with most music discovery along with unique URIs
sns_prep = streams_af_df.copy().query("(playtime / duration) < 0.25")
sns_prep.at[:, "count"] = 1
sns_prep = (
    sns_prep.loc[:, ["month", "duration", "playtime", "shuffle", "count"]]
    .groupby(by="month")
    .sum(numeric_only=True)
)
sns.barplot(data=sns_prep, x=sns_prep.index, y="count")


In [ ]:
# common keys of listened tracks
no_skip_df.loc[:, "timestamp"] = no_skip_df.loc[:, "timestamp"].astype(str)
pivot_track = pd.DataFrame(
    no_skip_df.drop_duplicates(subset="timestamp")
    .pivot_table(columns=["key_signature"], aggfunc="size")
    .sort_values(ascending=False)
)
pivot_track


In [ ]:
# def show_transformations(df, song, transformation="all"):
#     song_key = df.query("(id == @song)")["camelot"]
#     # print(song_key)
#     keys = wheel_df.loc[transformation, song_key].reset_index(drop=True)
#     # print(type(keys))
#     print(keys[0])
#     return df.query("camelot.isin(@keys[0])", engine="python")


# fuck = show_transformations(streams_af_df, "115RufHm0zvrPN3MaheZ98", ["major"])
# fuck
# # song_key = playlist_af_df.loc[1, "camelot"]
# # friendkey = wheel_df.loc["dominant_relative", song_key]
# # playlist_af_df.query("camelot in @friendkey")


In [ ]:
# # This version works with uri
# #should also have function to get uri from song title + artist
# #todo: proper type hinting and default values
# # separate functions i suppose, maybe with decorators
# # https://stackoverflow.com/questions/62153371/best-way-to-create-python-function-with-multiple-options


get_friendly(
    streams_af_df, tempo_range=10, uri="5jh3f8fy8a335XYDyJ7x9R", shift=["all"]
).sort_values(by="track")


In [ ]:
# How often do I play songs with "birthday" in the name on my loved ones' birthdays?
streams_af_df.query('(playtime / duration) > 0.1').query("track.str.contains('(?i)birthday')").sort_values(by=['month', 'date'])

In [ ]:
streams_df

,ts,ms_played,track,artist,album,id,start,end,shuffle,date,time,month,year
0,2021-03-25 18:43:26+00:00,351053,Throw Some D's Remix,Rich Boy,Rich Boy,1qXLj4ywjsA0hGmaibmnn3,trackdone,trackdone,False,03/25/2021,18:43:26,03,2021
1,2019-12-05 05:13:00+00:00,158360,Bounce,Latto,Big Latto,45anQ4WSf2a9QhZq55aaMw,trackdone,trackdone,False,12/05/2019,05:13:00,12,2019
2,2019-12-07 08:21:03+00:00,262800,"Can't Stop, Won't Stop",Young Gunz,Tough Luv,1kgDlypHy9hCvCxeoCtd7X,trackdone,trackdone,False,12/07/2019,08:21:03,12,2019
3,2021-09-17 03:23:34+00:00,44484,Douha (Mali Mali),Disclosure,ENERGY,33J9lmeViWXN0Le0vWRyjc,playbtn,endplay,False,09/17/2021,03:23:34,09,2021
4,2021-07-26 05:31:35+00:00,68341,You Got Me - Drum & Bass Mix,The Roots,Things Fall Apart,3aljYEbx7GiJraSrIu82kd,clickrow,endplay,False,07/26/2021,05:31:35,07,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85422,2022-01-27 00:42:30+00:00,22809,Nonstop Feeling,Mall Grab,Steel City Dance Discs Volume 20,2oQVKWHh7FpdPtQkqt3kMc,clickrow,endplay,False,01/27/2022,00:42:30,01,2022
85423,2020-10-11 19:29:48+00:00,263301,Next Levels,King Geedorah,Take Me To Your Leader,58MeeOMk6PMQxXWY9YCNPD,remote,trackdone,False,10/11/2020,19:29:48,10,2020
85424,2022-08-29 08:30:37+00:00,454490,The Frontier,Avalon Emerson,Whities 006,4qpY8nTiATIzPZ8eMucSID,trackdone,trackdone,True,08/29/2022,08:30:37,08,2022
85425,2021-10-03 07:36:02+00:00,580,La vita nuova,Christine and the Queens,La vita nuova,36LC0XouPxisV5wz8I0fZ9,fwdbtn,fwdbtn,True,10/03/2021,07:36:02,10,2021


In [ ]:
# Todo: plot podcast listening vs music listening
# damn i really spent most of january 2022 listening to podcats
podcasts_2020 = podcasts_df
podcasts_2020['playtime'] = podcasts_2020['playtime'] / 60000 / 60
podcast_per_month = podcasts_2020.groupby(by="month").sum(numeric_only=True)
podcast_per_month['playtime']
sns.barplot(data=podcast_per_month, x=podcast_per_month['playtime'], y=podcast_per_month.index)

In [ ]:
pod.loc[:, 'hours'] = podcast.loc[:, 'playtime']/3600000
pod.drop(columns=['spotify_episode_uri',
                            'start',
                           'playtime'])
pod.index = pod.ts
pod

podm = pod.resample('M').mean()

podm.index = podm.index.month
podm